# Histogram of power of theta in BLA during REM

In [1]:
import os

import bk.load
import bk.compute
import bk.plot
import bk.signal

import numpy as np

import scipy.signal
import scipy.stats
import scipy.io

import matplotlib.pyplot as plt
import matplotlib as mpl
import neuroseries as nts

In [2]:
%matplotlib qt

In [3]:
COLOR = 'white'
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['axes.edgecolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR
mpl.rcParams['figure.facecolor'] = '#282C34'
mpl.rcParams['axes.facecolor'] = '#282C34'

In [8]:
path = '/media/billel/GGLab-01-BK/DATA/Rat08-20130713/'
os.chdir(path)
bk.load.path = path
bk.load.rat = 8
bk.load.session = 'Rat08-20130713'
bk.load.day = 6

In [4]:
bk.load.current_session()

Rat : 8 on day : 6
Working with session Rat08-20130713 @ Z:\Rat08\Rat08-20130713


True

In [5]:
states = bk.load.states()
run = scipy.io.loadmat(bk.load.session + '-TrackRunTimes.mat')['trackruntimes']
run = nts.IntervalSet(np.round(run[:,0],decimals = 1),np.round(run[:,1],decimals = 1),time_units='s')
theta = {'Rem':states['Rem'],'run':run,'wake':states['wake']}

In [6]:
plt.figure()
plt.suptitle('Theta PhaseShift')
for i,state in enumerate(theta,1):
    
    lfp_hpc = bk.load.lfp_in_intervals(166,23,theta[state])
    lfp_bla = bk.load.lfp_in_intervals(166,105,theta[state])
    
    filt_hpc = bk.signal.passband(lfp_hpc,4,8)
    filt_bla = bk.signal.passband(lfp_bla,4,8)
    
    power_hpc,phase_hpc = bk.signal.hilbert(filt_hpc,deg = True)
    power_bla,phase_bla = bk.signal.hilbert(filt_bla,deg = True)
    
    plt.subplot(2,3,i)
    plt.hist2d(phase_hpc.values+180,phase_bla.values+180,100)
    plt.title(f'Theta Phase shift during {state}')
    plt.xlabel('hpc_phase')
    plt.ylabel('bla_phase')

In [14]:
# plt.figure()
# plt.suptitle('Spectrograms')

for i,state in enumerate(theta,4):
    
    lfp_hpc = bk.load.lfp_in_intervals(166,23,theta[state])
    lfp_bla = bk.load.lfp_in_intervals(166,105,theta[state])
    
    f,spec_hpc = scipy.signal.welch(lfp_hpc,fs = 1250,nperseg=10000,noverlap=9000)
    f,spec_bla = scipy.signal.welch(lfp_bla,fs = 1250,nperseg=10000,noverlap=9000)

    plt.subplot(2,3,i)
    plt.title(f'Power Spectrum during {state}')
    plt.plot(f,spec_hpc)
    plt.plot(f,spec_bla)
    plt.xlim(0,100)
    plt.yscale('log')
    plt.legend(['Hpc','BLA'])


In [11]:
t = np.arange(0,len(lfp_hpc),1)/1250

lfp_bla = bk.load.lfp_in_intervals(166,105,theta['run'])
lfp_hpc= bk.load.lfp_in_intervals(166,13,theta['run'])
plt.figure()
plt.suptitle('Example LFPs during RUN')
s = plt.subplot(2,1,1)
plt.title("Hpc")
plt.plot(t,lfp_hpc.values, c = 'azure',linewidth = 0.5)
plt.plot(t,bk.signal.passband(lfp_hpc,4,8).values,c = 'r')
plt.subplot(2,1,2,sharex=s,sharey = s)
plt.title('BLA')
plt.plot(t,lfp_bla.values, c = 'azure',linewidth = 0.5)
plt.plot(t,bk.signal.passband(lfp_bla,4,8).values,c = 'r')


In [9]:
plt.yscale('log')

In [28]:
lfp_hpc = bk.load.lfp_in_intervals(166,23,theta['run'])
filt_hpc = bk.signal.passband(lfp_hpc,4,8)
power_hpc,phase_hpc = bk.signal.hilbert(filt_hpc,deg = True)

In [29]:
lfp_fake = nts.Tsd(lfp_hpc.index.values,np.random.rand(len(lfp_hpc)))
filt_fake = bk.signal.passband(lfp_fake,4,8)
power_fake,phase_fake = bk.signal.hilbert(filt_fake,deg = True)

In [43]:
s = plt.subplot(2,1,1)
plt.plot(lfp_fake.index,lfp_fake.values-np.mean(lfp_fake.values),'azure')
plt.plot(filt_fake,'red')
plt.subplot(2,1,2,sharex = s)
plt.plot(phase_fake)

In [42]:
plt.hist2d(phase_hpc.values+180,phase_fake.values+180,100)

(array([[ 99.,  75.,  79., ...,  80.,  87., 101.],
        [ 85.,  91.,  92., ...,  88.,  68.,  95.],
        [ 98.,  84.,  87., ...,  95.,  84.,  74.],
        ...,
        [ 88.,  80.,  93., ...,  81.,  74.,  82.],
        [ 79.,  86.,  93., ..., 107.,  86.,  76.],
        [ 69.,  85.,  85., ...,  97., 105.,  82.]]),
 array([9.72203992e-04, 3.60096014e+00, 7.20094809e+00, 1.08009360e+01,
        1.44009240e+01, 1.80009119e+01, 2.16008998e+01, 2.52008878e+01,
        2.88008757e+01, 3.24008637e+01, 3.60008516e+01, 3.96008396e+01,
        4.32008275e+01, 4.68008154e+01, 5.04008034e+01, 5.40007913e+01,
        5.76007793e+01, 6.12007672e+01, 6.48007551e+01, 6.84007431e+01,
        7.20007310e+01, 7.56007190e+01, 7.92007069e+01, 8.28006948e+01,
        8.64006828e+01, 9.00006707e+01, 9.36006587e+01, 9.72006466e+01,
        1.00800635e+02, 1.04400622e+02, 1.08000610e+02, 1.11600598e+02,
        1.15200586e+02, 1.18800574e+02, 1.22400562e+02, 1.26000550e+02,
        1.29600538e+02, 1.33200

In [49]:
t = np.arange(0,3*np.pi,0.1)
y = np.sin(t)
y = nts.Tsd(t,y)

In [50]:
plt.plot(t,y)